In [ ]:
!pip install torchtext==0.4.0
!python -m spacy download en_core_web_sm

In [ ]:
pip install "numpy<2"

In [ ]:
import torch
from torchtext import data, datasets
import random

SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# TEXT field for tokenization
TEXT = data.Field(tokenize='spacy', 
                  tokenizer_language='en_core_web_sm', 
                  include_lengths=True)

# LABEL field for the multi-class classification labels 
LABEL = data.LabelField()

In [ ]:
# Load and split dataaset into the predefined training and test sets 
train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

# Split training data into train and validation sets 
train_data, valid_data = train_data.split(split_ratio=0.8, 
                                          stratified=True, 
                                          strata_field='label', 
                                          random_state=random.seed(SEED))
# Check data
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

# Print example
print(vars(train_data.examples[0]))

Part A 

In [ ]:
# Choose pre_trained_vectors
PRE_TRAINED_VECTORS = "glove.6B.300d" 

# Build vocabulary
TEXT.build_vocab(train_data, 
                 vectors=PRE_TRAINED_VECTORS, 
                 unk_init=torch.Tensor.normal_) 
#Q1(c) unk_init=torch.Tensor.normal_ initialized all words not found in GloVe (our OOV words) with random numbers from a normal distribution

LABEL.build_vocab(train_data)

Q1(a)

In [ ]:
# Vocabulary size
vocab_size = len(TEXT.vocab)
print(f"The size of the vocabulary from the training data is: {vocab_size}")

Q1(b)

In [ ]:
# Q1(b)

# Load the pre-trained vector dictionary manually to check for existence
from torchtext.vocab import GloVe
glove_vectors = GloVe(name='6B', dim=300)

# Count total OOV words
oov_words = []
for word in TEXT.vocab.itos:
    if word not in glove_vectors.stoi: 
        if word not in ['<unk>', '<pad>']:
            oov_words.append(word)

print(f"Total number of OOV words: {len(oov_words)}")

# Number of unique OOV words per topic category
import collections
oov_by_topic = collections.defaultdict(set)
for example in train_data.examples:
    topic = example.label
    for word in example.text:
        if word in oov_words:
            oov_by_topic[topic].add(word)

print("\nNumber of OOV words per topic category:")
for topic, words in oov_by_topic.items():
    print(f"{topic}: {len(words)}")

Q1(d)

In [ ]:
import nltk
from nltk.corpus import stopwords
import collections
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np  

stop_words = set(stopwords.words('english'))

topic_word_counts = collections.defaultdict(collections.Counter)
for ex in train_data.examples:
    topic = ex.label
    for word in ex.text:
        if word not in stop_words and word.isalpha(): # Check if alphabetic
            topic_word_counts[topic][word] += 1

top_words_data = []
for topic, counter in topic_word_counts.items():
    words_added = 0 
    for word, freq in counter.most_common(100): 
        
        if words_added >= 20:
            break
            
        if word in TEXT.vocab.stoi and word not in stop_words and word.isalpha():
            if word not in oov_words: 
                vector = TEXT.vocab.vectors[TEXT.vocab.stoi[word]].numpy()
                top_words_data.append({'word': word, 'topic': topic, 'vector': vector})
                words_added += 1 

df = pd.DataFrame(top_words_data)


vectors_for_tsne = np.array(list(df['vector']))

vectors_2d = TSNE(n_components=2, random_state=SEED).fit_transform(vectors_for_tsne)

df['x'] = vectors_2d[:, 0]
df['y'] = vectors_2d[:, 1]

plt.figure(figsize=(16, 12))
sns.scatterplot(data=df, x='x', y='y', hue='topic', legend='full', s=100) # s=size

# Add labels
for i, row in df.iterrows():
    plt.text(row['x'] + 0.1, row['y'], row['word'], fontsize=9)

plt.title('t-SNE Projection of Top 20 Words per Topic')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')